# ANNUITY CALCULATOR

In this notebook I genarate annuity cash flows.

In [1]:
from financepy.utils import *

In [2]:
from financepy.products.bonds import *
from financepy.utils import *
from financepy.products.rates import *

## CREATE AN ANNUITY STREAM

Calculate the cash flows on an annuity

In [3]:
value_dt = Date(20, 6, 2018)
settle_dt = value_dt

In [4]:
maturity_dt = settle_dt.add_tenor("5Y")  # Date(20, 9, 2028)
coupon = 0.05
freq_type = FrequencyTypes.SEMI_ANNUAL
cal_type = CalendarTypes.WEEKEND
bd_type = BusDayAdjustTypes.FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD
basis_type = DayCountTypes.ACT_360
face = ONE_MILLION

In [5]:
annuity = BondAnnuity(maturity_dt,coupon,freq_type,cal_type,
                         bd_type,dg_type,basis_type)

In [6]:
print(annuity)

OBJECT TYPE: BondAnnuity
MATURITY DATE: 20-JUN-2023
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
CALENDAR: CalendarTypes.WEEKEND
BUS_DAY_RULE: BusDayAdjustTypes.FOLLOWING
DATE_GEN_RULE: DateGenRuleTypes.BACKWARD



In [7]:
face = 1000000

In [8]:
annuity.print_payments(settle_dt, face)

20-DEC-2018 , 25416.666666666668
20-JUN-2019 , 25277.777777777777
20-DEC-2019 , 25416.666666666668
22-JUN-2020 , 25694.44444444444
21-DEC-2020 , 25277.777777777777
21-JUN-2021 , 25277.777777777777
20-DEC-2021 , 25277.777777777777
20-JUN-2022 , 25277.777777777777
20-DEC-2022 , 25416.666666666668
20-JUN-2023 , 25277.777777777777


## We can value the flows by discounting them

First we build a Libor swap curve

In [9]:
depos = []
dc_type = DayCountTypes.ACT_360
fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0500,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0500,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0500,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0500,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0500,fixed_freq,dc_type)
swaps = [swap1, swap2, swap3, swap4, swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

In [10]:
swap5.value(settle_dt, libor_curve)

5.820766091346741e-11

In [11]:
swap5.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 20-JUN-2023
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | RATE | PMNT     | DF     | PV       | CUM_PV   
--------+-------------+----------+------+----------+--------+----------+----------
      1 | 20-DEC-2018 |  1000000 |  5.0 | 25416.67 | 0.9752 | 24786.65 |  24786.65
      2 | 20-JUN-2019 |  1000000 |  5.0 | 25277.78 | 0.9512 | 24043.46 |  48830.11
      3 | 20-DEC-2019 |  1000000 |  5.0 | 25416.67 | 0.9276 | 23575.62 |  72405.74
      4 | 22-JUN-2020 |  1000000 |  5.0 | 25694.44 | 0.9044 | 23236.96 |   95642.7
      5 | 21-DEC-2020 |  1000000 |  5.0 | 25277.78 | 0.8821 | 22297.24 | 117939.94
      6 | 21-JUN-2021 |  1000000 |  5.0 | 25277.78 | 0.8603 | 21746.81 | 139686.75
      7 | 20-DEC-2021 |  1000000 |  5.0 | 25277.78 | 0.8391 | 21210.65 | 160897.39
      8 | 20-JUN-2022 |  1000000 |  5.0 | 25277.78 | 0.8184 | 20687.71 |  181585.1
      9 | 2

Here are the flows on the fixed leg of the 5Y swap

In [12]:
swap5.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 20-JUN-2023
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | IBOR   | PMNT     | DF     | PV       | CUM_PV   
--------+-------------+----------+--------+----------+--------+----------+----------
      1 | 20-SEP-2018 |  1000000 | 5.0794 | 12698.47 | 0.9875 | 12539.24 |  12539.24
      2 | 20-DEC-2018 |  1000000 | 5.0238 | 12559.58 | 0.9752 | 12248.26 |  24787.49
      3 | 20-MAR-2019 |  1000000 | 4.9683 |  12420.7 | 0.9632 | 11964.22 |  36751.71
      4 | 20-JUN-2019 |  1000000 | 5.0794 | 12698.47 | 0.9512 |  12078.4 |  48830.11
      5 | 20-SEP-2019 |  1000000 | 5.0854 |  12713.4 | 0.9392 |  11940.8 |  60770.91
      6 | 20-DEC-2019 |  1000000 | 5.0297 | 12574.35 | 0.9276 | 11663.53 |  72434.44
      7 | 20-MAR-2020 |  1000000 | 5.0177 | 12544.34 | 0.9161 | 11491.54 |  83925.98
      8 | 22-JUN-2020 |  1000000 | 5.0697 | 12955.84 | 0.9044 | 11716.7

In [13]:
cleanValue = annuity.clean_price_from_discount_curve(settle_dt, libor_curve)

In [14]:
dirtyValue = annuity.dirty_price_from_discount_curve(settle_dt, libor_curve)

The price of the annuity is quoted based on a par notional

In [15]:
print("CLEAN PRICE:", cleanValue)
print("DIRTY PRICE:", dirtyValue)
print("ACCRD VALUE:", annuity.accrued_int)

CLEAN PRICE: 22154839.385148082
DIRTY PRICE: 22154839.385148082
ACCRD VALUE: 0.0


Copyright (c) 2020 Dominic O'Kane